In [1]:
!pip install pykrx

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install langchain langchain[ollama] langchain[community]

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from pykrx import stock
from pykrx import bond

In [4]:
import json

def get_market_ohlcv(start_date:str, end_date:str, ticker:str):
    stock_name = stock.get_market_ticker_name(ticker)
    df = stock.get_market_ohlcv(fromdate=start_date, todate=end_date, ticker=ticker)
    df['종목명'] = [stock_name] * len(df)

    return json.dumps(df.to_dict(orient='records'), ensure_ascii=False)

In [5]:
print(get_market_ohlcv("20220720", "20220810", "005930"))

[{"시가": 61800, "고가": 62100, "저가": 60500, "종가": 60500, "거래량": 16782238, "등락률": -0.6568144499178982, "종목명": "삼성전자"}, {"시가": 61100, "고가": 61900, "저가": 60700, "종가": 61800, "거래량": 12291374, "등락률": 2.1487603305785123, "종목명": "삼성전자"}, {"시가": 61800, "고가": 62200, "저가": 61200, "종가": 61300, "거래량": 10261310, "등락률": -0.8090614886731391, "종목명": "삼성전자"}, {"시가": 60900, "고가": 61900, "저가": 60800, "종가": 61100, "거래량": 9193681, "등락률": -0.3262642740619902, "종목명": "삼성전자"}, {"시가": 60800, "고가": 61900, "저가": 60800, "종가": 61700, "거래량": 6597211, "등락률": 0.9819967266775778, "종목명": "삼성전자"}, {"시가": 61300, "고가": 61900, "저가": 61200, "종가": 61800, "거래량": 7320997, "등락률": 0.1620745542949757, "종목명": "삼성전자"}, {"시가": 62300, "고가": 62600, "저가": 61600, "종가": 61900, "거래량": 10745302, "등락률": 0.16181229773462785, "종목명": "삼성전자"}, {"시가": 62400, "고가": 62600, "저가": 61300, "종가": 61400, "거래량": 15093120, "등락률": -0.8077544426494345, "종목명": "삼성전자"}, {"시가": 61000, "고가": 61700, "저가": 60300, "종가": 61300, "거래량": 13154816, "등락률": -0.1628664495114

In [6]:
from langchain_core.tools import tool

@tool
def get_market_ohlcv_tool(start_date, end_date, ticker):
    """Returns stock price data for a set period of time for a particular stock. start_date and end_date should be 'YYYYMMDD' format string and ticker should be 6-digits ."""
    start_date = str(start_date) if isinstance(start_date, int) else start_date
    end_date = str(end_date) if isinstance(end_date, int) else end_date
    return get_market_ohlcv(start_date, end_date, ticker)

In [ ]:
#from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_ollama import ChatOllama

import os

llm = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"), format="json", temperature=0)
llm_with_tools = llm.bind_tools(tools=[get_market_ohlcv_tool])

response = llm_with_tools.invoke("안녕?")
print(response)

query = "LG전자(066570)의 2022년 8월 1일부터 2022년 9월 1일까지의 주가 데이터를 요약해서 의견을 줘."
response = llm_with_tools.invoke(query)
print(response)

content='' additional_kwargs={} response_metadata={} id='run-3e372014-0801-45a1-bc21-08c33ab6ea65-0' tool_calls=[{'name': 'directly-answer', 'args': {}, 'id': '815c3624-6378-430e-b7ab-cf90da3caf25', 'type': 'tool_call'}]
content='' additional_kwargs={} response_metadata={} id='run-8cb13827-229e-450d-96bc-c6e3a8a76273-0' tool_calls=[{'name': 'get_market_ohlcv_tool', 'args': {'end_date': 20220901, 'start_date': 20220801, 'ticker': '066570'}, 'id': '96929156-cb2e-44e0-b873-7f26672aafd6', 'type': 'tool_call'}]


In [8]:
from langchain_core.messages import HumanMessage, FunctionMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(query)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_market_ohlcv_tool": get_market_ohlcv_tool}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(FunctionMessage(tool_output, name=tool_call["name"]))

print(messages)

[HumanMessage(content='LG전자(066570)의 2022년 8월 1일부터 2022년 9월 1일까지의 주가 데이터를 요약해서 의견을 줘.', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={}, response_metadata={}, id='run-94e1a50e-9bbb-481e-953f-f65fb2a095b7-0', tool_calls=[{'name': 'get_market_ohlcv_tool', 'args': {'end_date': 20220901, 'start_date': 20220801, 'ticker': '066570'}, 'id': '278b3b26-74fd-43a6-89f7-c9bac1e7c184', 'type': 'tool_call'}]), FunctionMessage(content='[{"시가": 94800, "고가": 95300, "저가": 93600, "종가": 94400, "거래량": 485739, "등락률": 0.0, "종목명": "LG전자"}, {"시가": 94400, "고가": 95000, "저가": 93200, "종가": 93500, "거래량": 474933, "등락률": -0.9533898305084746, "종목명": "LG전자"}, {"시가": 93500, "고가": 93900, "저가": 93000, "종가": 93800, "거래량": 374218, "등락률": 0.32085561497326204, "종목명": "LG전자"}, {"시가": 94400, "고가": 96900, "저가": 94200, "종가": 96000, "거래량": 1059583, "등락률": 2.345415778251599, "종목명": "LG전자"}, {"시가": 96400, "고가": 96700, "저가": 95400, "종가": 95900, "거래량": 447686, "등락률": -0.10416666666666667, "종목명":

In [9]:
from langchain_community.llms import Ollama
import os

Ollama(model=os.getenv("OPENAI_DEFAULT_MODEL"), temperature=0).invoke(messages)

/tmp/ipykernel_13969/1621789022.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  Ollama(model=os.getenv("OPENAI_DEFAULT_MODEL"), temperature=0).invoke(messages)


'제공된 데이터는 LG전자의 주가 변동 기록을 보여줍니다. 기록은 다양한 시가와 가격 범위, 거래량, 그리고 등락률을 포함합니다. 주요 포인트는 다음과 같습니다:\n\n- **가격 변동성:** 주가는 시간에 따라 크게 변동하는 것을 알 수 있습니다. 예를 들어, 일부 데이터 포인트에서는 4% 이상의 급격한 상승이나 하락이 나타납니다.\n- **최고 및 최저 가격:** 가장 높은 종가는 102,500원, 가장 낮은 종가는 97,700원입니다.\n- **거래량:** 거래량은 주가의 변동성과 비례하는 경향을 보이며, 높은 가격 변동성이 있는 기간에 거래량이 증가합니다.\n- **등락률:** 등락률은 주가의 변화 정도를 나타내며, 일부 날에는 -4% 이상의 하락도 기록됩니다.\n\n이 데이터는 LG전자의 주식 성과를 분석하고, 투자 결정에 참고할 수 있는 중요한 지표를 제공합니다.'

In [10]:
from langchain_ollama import ChatOllama
import os

llm = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"), format="json", temperature=0)
llm_with_tools = llm.bind_tools(tools=[get_market_ohlcv_tool])

messages = [
    HumanMessage(
        content = "LG전자(066570)의 2022년 8월 1일부터 2022년 9월 1일까지의 주가 데이터를 요약해서 의견을 줘."
    )
]
response = llm_with_tools.invoke(messages)
print(response)

content='' additional_kwargs={} response_metadata={} id='run-3db86055-536d-4a04-8a0b-5d58cbc969ce-0' tool_calls=[{'name': 'get_market_ohlcv_tool', 'args': {'end_date': 20220901, 'start_date': 20220801, 'ticker': '066570'}, 'id': '915fee48-44f6-43b5-8540-3de69eb6261d', 'type': 'tool_call'}]


In [11]:
from langchain_ollama import ChatOllama
import os

llm = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"), format="json", temperature=0)
llm_with_tools = llm.bind_tools(tools=[get_market_ohlcv_tool])

messages = [
    HumanMessage(
        content = "LG전자(066570)의 2022년 8월 1일부터 2022년 9월 1일까지의 주가 데이터를 요약해서 의견을 줘."
    )
]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_market_ohlcv_tool": get_market_ohlcv_tool }[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(FunctionMessage(tool_output, name=tool_call["name"]))
    
messages

[HumanMessage(content='LG전자(066570)의 2022년 8월 1일부터 2022년 9월 1일까지의 주가 데이터를 요약해서 의견을 줘.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={}, id='run-4db06001-26f0-484e-8f45-6f327f8496b7-0', tool_calls=[{'name': 'get_market_ohlcv_tool', 'args': {'end_date': 20220901, 'start_date': 20220801, 'ticker': '066570'}, 'id': 'e7694d89-c1fa-4753-985f-2d916c9eee46', 'type': 'tool_call'}]),
 FunctionMessage(content='[{"시가": 94800, "고가": 95300, "저가": 93600, "종가": 94400, "거래량": 485739, "등락률": 0.0, "종목명": "LG전자"}, {"시가": 94400, "고가": 95000, "저가": 93200, "종가": 93500, "거래량": 474933, "등락률": -0.9533898305084746, "종목명": "LG전자"}, {"시가": 93500, "고가": 93900, "저가": 93000, "종가": 93800, "거래량": 374218, "등락률": 0.32085561497326204, "종목명": "LG전자"}, {"시가": 94400, "고가": 96900, "저가": 94200, "종가": 96000, "거래량": 1059583, "등락률": 2.345415778251599, "종목명": "LG전자"}, {"시가": 96400, "고가": 96700, "저가": 95400, "종가": 95900, "거래량": 447686, "등락률": -0.10416666666666667, "종목명

In [12]:
from langchain_community.llms import Ollama
import os

Ollama(model=os.getenv("OPENAI_DEFAULT_MODEL"), temperature=0).invoke(messages)

'제공된 데이터는 LG전자의 주가 변동 기록을 보여줍니다. 기록은 다양한 시가와 가격 범위, 거래량, 그리고 등락률을 포함합니다. 주요 포인트는 다음과 같습니다:\n\n- **가격 변동성:** 주가는 9만 원 대부터 10만 원 대까지 다양하게 변동하며 높은 가격 변동성을 보입니다.\n- **최고 및 최저 기록:** 가장 높은 종가는 102,500원, 가장 낮은 종가는 97,700원입니다.\n- **거래량:** 거래량은 50만 주에서 800만 주 이상까지 다양하며, 주가의 변동성과 관련이 있는 것으로 보입니다.\n- **등락률:** 하루 동안의 등락률은 -4.27%에서 3.59%까지 다양합니다.\n\n이 데이터는 LG전자의 주식 성과를 분석하고, 투자 결정에 참고할 수 있는 중요한 정보를 제공합니다.'